In [1]:
import pandas as pd
import json

caminho_arquivo = 'D:\VSCode\dados\data.csv'

dados = pd.read_csv(caminho_arquivo, on_bad_lines='skip', sep=',(?=\S)', engine='python')

# Tirando as aspas

def delete_quotes(x):
    return x[1:-1]
for column in ["id", "screenName", "avatar", "lang", "tweetId"]:
    dados[column] = dados[column].apply(delete_quotes)
    
for column in ["tags", "friends"]:
    dados[column] = dados[column].apply(lambda x: json.loads(x))

# Dropando colunas inuteis
dados = dados.drop(columns=["avatar", "followersCount", "friendsCount", "lang", "lastSeen", "tweetId"])

In [23]:
# Cortando somente 1000 entradas do dataset

# Verificar o número total de linhas do dataset
total_linhas = len(dados)

# Definir o número desejado de linhas a serem mantidas
n_linhas_desejadas = 1000

# Verificar se o número desejado de linhas é menor que o número total de linhas
if n_linhas_desejadas < total_linhas:
    # Determinar o número de linhas que serão removidas
    n_linhas_removidas = total_linhas - n_linhas_desejadas

    # Remover as linhas aleatoriamente
    dados = dados.sample(n=n_linhas_desejadas, random_state=42)

In [2]:
# Pegando menos usuarios para agilizar
df_reduzido = dados.head(100)

# Encontre todos os IDs de amigos
all_friends = set()
for friends in df_reduzido['friends']:
    all_friends.update(set(friends))

# Crie um dicionário para contar a ocorrência de cada ID de amigo
friend_counts = {friend: 0 for friend in all_friends}
for friends in df_reduzido['friends']:
    for friend in friends:
        friend_counts[friend] += 1

# Remova os IDs de amigos que não estão presentes em outras listas de amigos
def remove_unique_friends(friends):
    return [friend for friend in friends if friend_counts[friend] > 1]

df_copy = df_reduzido.copy()

df_copy['friends'] = df_copy['friends'].apply(remove_unique_friends)

# Remova os usuários que não têm amigos em comum com pelo menos um outro usuário
df_copy = df_copy.loc[df_copy['friends'].apply(lambda x: len(x) > 0)]

# Gera o CSV
df_copy.to_csv('D:\VSCode\dados\dados100.csv', index=False)

In [3]:
# CÓDIGO PARA GERAR O CSV DE CONTAS SEGUIDAS

# Cria uma lista com todos os IDs
all_ids = []
for friends_list in df_copy['friends']:
    all_ids.extend(friends_list)

# Remove duplicatas e cria o dataframe com a coluna 'id_follower' contendo todos os IDs
df_id_follower = pd.DataFrame({'id_follower': list(set(all_ids))})

df_id_follower.to_csv('D:\VSCode\dados\dadosFollowers100.csv', index=False)